<a href="https://colab.research.google.com/github/cristiano-silva-0182/datasets_estudo/blob/main/compila%C3%A7%C3%A3o_vacina%C3%A7%C3%A3o_SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

In [8]:
# FUNÇÃO PARA A CORREÇÃO DE INCONSISTÊNCIAS DE NOMES DE MUNICÍPIOS
def ajuste_cidades(x):
  x['cidade'] = x['cidade'].replace("''","'", regex=True)

  x.loc[x.iloc[:,0]=='ARCO IRIS',x.columns[0]] =  'ARCO-IRIS'
  x.loc[x.iloc[:,0]=='BIRITIBA-MIRIM',x.columns[0]] =  'BIRITIBA MIRIM'
  x.loc[x.iloc[:,0]=='EMBU GUACU',x.columns[0]] =  'EMBU-GUACU'
  x.loc[x.iloc[:,0]=='Estado - Sao Paulo',x.columns[0]] =  'SAO PAULO'
  x.loc[x.iloc[:,0]=='FLORINIA',x.columns[0]] =  'FLORINEA'
  x.loc[x.iloc[:,0]=='LUIS ANTONIO',x.columns[0]] =  'LUIZ ANTONIO'
  x.loc[x.iloc[:,0]=='PARIQUERA ACU',x.columns[0]] =  'PARIQUERA-ACU'
  x.loc[x.iloc[:,0]=='SANTA RITA DO PASSA QUATR..',x.columns[0]] =  'SANTA RITA DO PASSA QUATRO'
  x.loc[x.iloc[:,0]=='SANTO ANTONIO DO ARACAN..',x.columns[0]] =  'SANTO ANTONIO DO ARACANGUA'
  x.loc[x.iloc[:,0]=="SAO JOAO DO PAU D'ALHO",x.columns[0]] =  "SAO JOAO DO PAU-D'ALHO"
  x.loc[x.iloc[:,0]=='SAO LUIS DO PARAITINGA',x.columns[0]] =  'SAO LUIZ DO PARAITINGA'

  x.loc[x.iloc[:,1]=='1 DOSE',x.columns[1]] =  'D1'
  x.loc[x.iloc[:,1]=='1 Dose',x.columns[1]] =  'D1'
  x.loc[x.iloc[:,1]=='2 DOSE',x.columns[1]] =  'D2'
  x.loc[x.iloc[:,1]=='2 Dose',x.columns[1]] =  'D2'
  x.loc[x.iloc[:,1]=='3o DOSE',x.columns[1]] =  'D3'
  x.loc[x.iloc[:,1]=='3o DOSE/ADICIONAL',x.columns[1]] =  'D3'
  

In [9]:
# CARREGA DATAFRAME DE MUNICÍPIOS BR
df_municipios = None
df_municipios = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/cristiano-silva-0182/datasets_estudo/main/00_dados_csv/municipios_BR.csv", sep=";"))
df_municipios['mun_desc_sem_acentuacao'] = df_municipios['mun_desc_sem_acentuacao'].str.upper()
df_municipios['mun_desc'] = df_municipios['mun_desc'].str.upper()
df_municipios['mun_id'] = pd.to_numeric(df_municipios['mun_id'], downcast='integer')
df_municipios = df_municipios.loc[(df_municipios['uf']=='SP')]

In [10]:
# Definição de variáveis
hoje = date.today() + relativedelta(days=-2)
d1 = date(2021,1,16)
d2 = date(hoje.year,hoje.month,(hoje.day))
dif_dias = int(abs(d2-d1).days)+1
prefixo ='https://raw.githubusercontent.com/cristiano-silva-0182/base_distribuicao_e_aplicacao/main/aplica%C3%A7%C3%A3o/'
sufixo ='_vacinometro.csv'

# Montagem de tabela Calendário
tabela_calendario = []
for n in range(dif_dias):
  data = hoje + relativedelta(days=-n)
  data = data.strftime("%Y-%m-%d")
  tabela_calendario.append(data)

df= pd.DataFrame(tabela_calendario, columns =["data"])
df['data'] = pd.to_datetime(df['data'])
df['ano']  = df['data'].dt.strftime('%Y')
df['mes'] = df['data'].dt.strftime('%m')
df['dia'] = df['data'].dt.strftime('%d')
df['url'] = prefixo+df['ano']+df['mes']+df['dia']+sufixo


In [11]:
# Lista Datas
lista_datas = list(df['data'])
lista_enderecos = list(df['url'])

df_compilado = pd.DataFrame()
colunas_dados = ['cidade',
               'dose',
               'qtde',
               'data']

# Montagem de Dataframe
df_inicial=pd.DataFrame()
for dt, endereço in zip(lista_datas, lista_enderecos):
    try:
        url = endereço
        df_arquivo = pd.read_csv(url,sep=';')
        df_arquivo = df_arquivo.assign(data = dt)
        df_arquivo.columns = colunas_dados
        df_arquivo = pd.DataFrame(df_arquivo)
        df_arquivo = df_arquivo.astype({
            'cidade':str,
            'dose':str,
            'qtde':int})
        
        df_inicial = df_inicial.append(df_arquivo, ignore_index=True)
        df_arquivo = []

    except:
        pass

df_cidades = pd.DataFrame(df_inicial)

cols = df_cidades.select_dtypes(include = [np.object]).columns
df_cidades[cols] = df_cidades[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8'))
#Corrige inconsistências no campo 'municipio_resid'
df_cidades['cidade'] = df_cidades['cidade'].str.strip() 
ajuste_cidades(x=df_cidades)
#Carrega as informações de Código IBGE e UF
df_cidades = pd.merge(df_cidades, df_municipios, left_on="cidade", right_on="mun_desc_sem_acentuacao").drop('mun_desc_sem_acentuacao', axis=1)
columns = ['regiao', 'uf', 'estado','area','pop_2010','porte','cidade']
df_cidades.drop(columns, inplace=True, axis=1)
df_cidades